In [4]:
import spacy
from spacy import displacy
from IPython.display import HTML, display
import pandas as pd

nlp = spacy.load("fr_core_news_sm")


TEXT = "Le chat mange une souris."

doc = nlp(TEXT)

In [5]:
token_infos_lines = []

for token in doc:
    token_infos_lines.append((token.text, token.lemma_, token.pos_, token.dep_,  token.idx, token.head, token.head.idx))

In [6]:
displacy.render(doc, style="dep", jupyter=True)

ImportError: cannot import name 'display' from 'IPython.core.display' (/Users/pernebayarailym/Documents/Portfolio_Projects_AP/Simplon_DE_Projects/Python_Projects/Neo4J_Spacy_Data_mining/.venv/lib/python3.13/site-packages/IPython/core/display.py)

In [ ]:
pd.DataFrame(token_infos_lines, columns=['text', 'lemma', 'pos','dep', 'idx',  'head', 'head_idx'])

,text,lemma,pos,dep,idx,head,head_idx
0,Le,le,DET,det,0,chat,3
1,chat,chat,NOUN,nsubj,3,mange,8
2,mange,manger,VERB,ROOT,8,mange,8
3,une,un,DET,det,14,souris,18
4,souris,souris,NOUN,obj,18,mange,8
5,.,.,PUNCT,punct,24,mange,8


In [ ]:
# Autre exemple, on peut séparer logiquement le texte par phrase :

TEXT2 = "Le chat dort sur le canapé. Le chien dort dans sa niche."
doc = nlp(TEXT2)

for sentence in doc.sents:
    print(sentence)

Le chat dort sur le canapé.
Le chien dort dans sa niche.




Si vous avez cette erreur :

https://github.com/explosion/spacy/issues/13864

avec 
```
displacy.render(doc, style="dep", jupyter=True)
```

changez dans le fichier incriminé :
```
from IPython.core.display import HTML, display
```

par : 
```
from IPython.display import HTML, display
```

# PERSONAL NOTE
Create a venv (in a folder called .venv)
```
python3.13 -m venv .venv
```
Activate it
```
source .venv/bin/activate
```

Install spacy inside the venv
```
pip install -U pip
pip install spacy
```

install ipykernel inside venv
```
pip install ipykernel
```

add venv as a Jupyter kernel
```
python -m ipykernel install --user --name=myvenv --display-name "Python (myvenv)"
```